In [ ]:
import numpy as np 
from qTools import *

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import datetime

# Define Input Parameters

In [ ]:
wq = 1
wc = 1
dim = 2
g = 1

In [ ]:
system = Qubit(frequency=wq) + Cavity(dimension=dim, frequency=wc)

coupling = system.JC(g)

system.initialState = identity(system.dimension)

In [ ]:
halfTime = 5
system.simulation.totalTime = 2*halfTime
system.simulation.stepCount = 150
system.simulation.delStates = True

In [ ]:
Amax = 1.5
Amin = 0.5
Bmax = 2.0
Bmin = 0.5
NA = 60 #Steps in Amplitude array
NB = 60 #Steps in Standard deviation array

AA = np.linspace(Amin, Amax, NA)
BB = np.linspace(Bmin, Bmax, NB)

ampSweep = system.simulation.Sweep.createSweep(system='Qubit1', sweepKey='amp', sweepList = AA)
bSweep = system.simulation.Sweep.createSweep(system='Qubit1', sweepKey='std',  sweepList = BB, multiParam=True)

timeDependency = system.simulation.timeDependency.createSweep(system='Qubit1')

def specialSweep(sweep):
    #compSys = sweep.getObjByName('QuantumSystem1')

    time = sweep.system.simulation.stepSize*sweep.index
    amplitude = sweep.system.aux['amp']
    std = sweep.system.aux['std']

    sweep.system.frequency = amplitude*np.exp(-(time-halfTime)**2/std**2)

timeDependency.sweepFunction = specialSweep

In [ ]:
def compute(qSim, *args):
    qSim.qRes.result = ('qubit freq', qSim.getObjByName('Qubit1').frequency)

def calculate(sys):
    res = sys.qRes
    unitary = sys.unitary

    res.result = ('00real', np.real(unitary[0,0]))
    res.result = ('11real', np.real(unitary[1,1]))
    res.result = ('22real', np.real(unitary[2,2]))
    res.result = ('33real', np.real(unitary[3,3]))

    res.result = ('00imag', np.imag(unitary[0,0]))
    res.result = ('11imag', np.imag(unitary[1,1]))
    res.result = ('22imag', np.imag(unitary[2,2]))
    res.result = ('33imag', np.imag(unitary[3,3]))

    res.result = ('03real', np.real(unitary[0,3]))
    res.result = ('30real', np.real(unitary[3,0]))

    res.result = ('03imag', np.imag(unitary[0,3]))
    res.result = ('30imag', np.imag(unitary[3,0]))

system.calculate = (calculate, 'end')
system.simulation.compute = compute

nw = datetime.datetime.now()

system.simulation.run(p=True, coreCount='all')

en = datetime.datetime.now()
print(en-nw)

In [ ]:
print(np.abs(system.unitary.A))

In [ ]:
print(system.simulation.qRes.results['qubit freq'][0][0])

In [ ]:
time = [system.simulation.stepSize*x for x in range(system.simulation.stepCount-1)]
print(time)

In [ ]:
plt.plot(time, system.simulation.qRes.results['qubit freq'][0][0])
plt.plot(time, system.simulation.qRes.results['qubit freq'][29][0])
plt.plot(time, system.simulation.qRes.results['qubit freq'][59][0])
plt.plot(time, system.simulation.qRes.results['qubit freq'][0][29])
plt.plot(time, system.simulation.qRes.results['qubit freq'][0][29])
plt.plot(time, system.simulation.qRes.results['qubit freq'][15][15])

In [ ]:
def pltcolor(axis, X, Y, Z):
    surf = axis.pcolormesh(Y,X,Z,  cmap='plasma') #vmin =-1, vmax=1
    divider = make_axes_locatable(axis)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(surf, cax=cax, orientation='vertical')
    axis.set_xlabel('Gauss std "B" variation (time)')
    axis.set_ylabel('Amp "A" Variation (Frequency)')


Y, X = np.meshgrid(BB,AA)


fig, ((ax1,ax7), (ax2,ax8), (ax3,ax9), (ax4,ax10), (ax5,ax11), (ax6,ax12)) = plt.subplots(nrows=6, ncols=2, figsize=(12,15))
pltcolor(ax1,X,Y, system.qRes.results['00real'])
ax1.set_title('Real Element [0,0]')
pltcolor(ax2,X,Y, system.qRes.results['11real'])
ax2.set_title('Real Element [1,1]')
pltcolor(ax3,X,Y, system.qRes.results['22real'])
ax3.set_title('Real Element [2,2]')
pltcolor(ax4,X,Y, system.qRes.results['33real'])
ax4.set_title('Real Element [3,3]')
pltcolor(ax5,X,Y, system.qRes.results['03real'])
ax5.set_title('Real Element [0,3]')
pltcolor(ax6,X,Y, system.qRes.results['30real'])
ax6.set_title('Real Element [3,0]')
pltcolor(ax7,X,Y, system.qRes.results['00imag'])
ax7.set_title('Imaginary Element [0,0]')
pltcolor(ax8,X,Y, system.qRes.results['11imag'])
ax8.set_title('Imaginary Element [1,1]')
pltcolor(ax9,X,Y, system.qRes.results['22imag'])
ax9.set_title('Imaginary Element [2,2]')
pltcolor(ax10,X,Y, system.qRes.results['33imag'])
ax10.set_title('Imaginary Element [3,3]')
pltcolor(ax11,X,Y, system.qRes.results['03imag'])
ax11.set_title('Imaginary Element [0,3]')
pltcolor(ax12,X,Y, system.qRes.results['30imag'])
ax12.set_title('Imaginary Element [3,0]')

plt.tight_layout()

In [ ]:
NT = 150 #Time Steps
Tf = 10.0 #Total time
Ti = 0.0 #Starting time
dt = (Tf-Ti)/NT 
t = np.linspace(0,Tf,NT) 
Tm = (Tf-Ti)/2
wq = 1
wc = 1
dim = 2
g = 1e-3

In [ ]:
#Parameters for square function
b1 = 4
b2 = 6

In [ ]:
#Paramters for Gaussian
Amax = 1.5
Amin = 0.5
Bmax = 2.0
Bmin = 0.5
NA = 150 #Steps in Amplitude array
NB = 150 #Steps in Standard deviation array
AA = np.linspace(Amin,Amax,NA) #Amplitude array
BB = np.linspace(Bmin,Bmax,NB) #Std array

# Define Functions

In [ ]:
def gauss(time, std, amplitude):
    G = amplitude*np.exp(-(time-Tm)**2/std**2)
    return G

def square(time,boundary_1,boundary_2):
    S = wq*(time>boundary_1)*(time<boundary_2)
    return S

def plot_graph(ax, xvalues, yvalues, fontsize=12):
    ax.plot(xvalues,yvalues)

    ax.locator_params(nbins=3)
    ax.set_xlabel('Time', fontsize=fontsize)
    ax.set_ylabel('Qubit Frequnecy (Hz)', fontsize=fontsize)
    ax.set_title('Gaussian VS Square', fontsize=fontsize)
    ax.set_ylim(0,1.6)

def Ugauss(time, std, amplitude):
    qub.frequency = gauss(time,std,amplitude)
    H = system.totalHam.A
    Ugauss = sy.expm(-1j*H*dt)
    return Ugauss

def Usqaure(time, boundary1, boundary2):
    qub.frequency = square(time,boundary1,boundary2)
    H = system.totalHam.A
    Usquare = sy.expm(-1j*H*dt)
    return Usquare

def fetch_realZ(Element_1, Element_2):
    z = np.zeros([NA,NB])
    for ii in range(len(U)):
        for jj in range(len(U)):
            z[ii,jj] = np.real(U[ii][jj][Element_1, Element_2])
    return z

def fetch_imagZ(Element_1, Element_2):
    z = np.zeros([NA,NB])
    for ii in range(len(U)):
        for jj in range(len(U)):
            z[ii,jj] = np.imag(U[ii][jj][Element_1, Element_2])
    return z

def pltcolor(axis, X, Y, Z):
    surf = axis.pcolormesh(Y,X,Z,  cmap='plasma') #vmin =-1, vmax=1
    divider = make_axes_locatable(axis)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(surf, cax=cax, orientation='vertical')
    axis.set_xlabel('Gauss std "B" variation (time)')
    axis.set_ylabel('Amp "A" Variation (Frequency)')

def makeDir(RootPath):
    now = datetime.now()
    date_time = now.strftime("%Y-%m-%d")
    path = RootPath + date_time
    if not os.path.isdir(path):
        os.mkdir(path)
    return path + '/' + str(now.timestamp())

save_path = makeDir(save_folder)

# Sanity Checks - Limitations

In [ ]:
fig, ((ax1, ax2, ax3, ax4)) = plt.subplots(nrows=1, ncols=4, figsize=(16,4))
plot_graph(ax1, t, gauss(t,Bmax,Amax))
plot_graph(ax1, t, square(t,b1,b2))
plot_graph(ax2, t, gauss(t,Bmin,Amax))
plot_graph(ax2, t, square(t,b1,b2))
plot_graph(ax3, t, gauss(t,Bmax,Amin))
plot_graph(ax3, t, square(t,b1,b2))
plot_graph(ax4, t, gauss(t,Bmin,Amin))
plot_graph(ax4, t, square(t,b1,b2))

plt.tight_layout()

# Define System

In [ ]:
system = QuantumSystem()
cav = Cavity(dimension=dim, frequency=wc)    
qub = Qubit()
system.add(qub, cav)
coupling = system.createSysCoupling([qub,cav], [destroy, create], couplingStrength=g)
coupling.addTerm([qub,cav],[create, destroy])
system.constructCompSys()

Uinitial = np.identity(dim*2)

# Calculating total Unitary

In [ ]:
#just for practise runs
'''
Utotal = []
for A in AA:
    UA = []
    for B in BB:
        U = Uinitial
        for kk in t:
            Udt = Ugauss(kk, B, A)
            U = Udt@U
        UA.append(U)
    Utotal.append(UA)

#remember enumerate 'for ii,A in enumerate(AA)
'''


# Retrieving Data

In [ ]:
filename = "C:/Users/andre/OneDrive/Documents/QFYZ/Jaynes-Cummings_Model/QuantumSimulations/Data/2020-05-27/1590565140.486998.h5"

f = h5py.File(filename, 'r')

U = np.array(f['Data']['Total Unitary'])

# Plotting Real and Imaginary Elements

In [ ]:
Y, X = np.meshgrid(BB,AA)

fig, ((ax1,ax7), (ax2,ax8), (ax3,ax9), (ax4,ax10), (ax5,ax11), (ax6,ax12)) = plt.subplots(nrows=6, ncols=2, figsize=(12,15))
pltcolor(ax1,X,Y,fetch_realZ(0,0))
ax1.set_title('Real Element [0,0]')
pltcolor(ax2,X,Y,fetch_realZ(1,1))
ax2.set_title('Real Element [1,1]')
pltcolor(ax3,X,Y,fetch_realZ(2,2))
ax3.set_title('Real Element [2,2]')
pltcolor(ax4,X,Y,fetch_realZ(3,3))
ax4.set_title('Real Element [3,3]')
pltcolor(ax5,X,Y,fetch_realZ(1,2))
ax5.set_title('Real Element [1,2]')
pltcolor(ax6,X,Y,fetch_realZ(2,1))
ax6.set_title('Real Element [2,1]')
pltcolor(ax7,X,Y,fetch_imagZ(0,0))
ax7.set_title('Imaginary Element [0,0]')
pltcolor(ax8,X,Y,fetch_imagZ(1,1))
ax8.set_title('Imaginary Element [1,1]')
pltcolor(ax9,X,Y,fetch_imagZ(2,2))
ax9.set_title('Imaginary Element [2,2]')
pltcolor(ax10,X,Y,fetch_imagZ(3,3))
ax10.set_title('Imaginary Element [3,3]')
pltcolor(ax11,X,Y,fetch_imagZ(1,2))
ax11.set_title('Imaginary Element [1,2]')
pltcolor(ax12,X,Y,fetch_imagZ(2,1))
ax12.set_title('Imaginary Element [2,1]')

plt.tight_layout()
plt.savefig(save_path+'.png')

# Saving Data

In [ ]:
# .txt file
with open(save_path + '.txt', 'w') as f:
    f.write('Explanation \n')
    f.write('Cavity frequency: ' + str(cav.frequency) + '\n')
    f.write('Qubit frequency: ' + str(qub.frequency) + '\n')
    f.write('Total time: ' + str(Tf) + '\n')
    f.write('Time Steps: ' + str(NT) + '\n')
    f.write('Gaussian Amplitude:' + str(AA) +'\n')
    f.write('Gaussian Standard deviation:' + str(BB) +'\n')
    f.write('Total unitary:' + str(Utotal) +'\n')
f.close()

# h5 file
file = h5py.File(save_path + '.h5', 'w')

parameters = file.create_group('Parameters')
parameters.create_dataset('omegaC', data=cav.frequency)
parameters.create_dataset('omegaQ', data=qub.frequency)
parameters.create_dataset('t_total', data=Tf)
parameters.create_dataset('t_steps', data=NT)
parameters.create_dataset('Amp', data=AA)
parameters.create_dataset('GStd', data=BB)
parameters.create_dataset('total Unitary', data = Utotal)

sweep = file.create_group('Sweep')
sweep.create_dataset('time', data=t)

result = file.create_group('Data')
result.create_dataset('Total Unitary', data=Utotal)
file.close()

In [ ]:
file.close()